In [2]:
import os
import json
import pandas as pd
from pandas import json_normalize

In [3]:
MODE = 'eval' #eval or predict (inference)

In [4]:
#Prepare the labels in coco format
paths=['downloads/SoccerPass/SoccerPass-COCO/val.json']

dfs= []
for path in paths:
    # Load the COCO data from the JSON file
    with open(path) as f:
        coco_data = json.load(f)

    # Normalize the data into a DataFrame
    images_df = json_normalize(coco_data['images'])
    annotations_df = json_normalize(coco_data['annotations'])

    images_df=images_df.drop(columns=['license', 'flickr_url','coco_url', 'date_captured'])
    annotations_df=annotations_df.drop(columns=['segmentation', 'area','attributes.occluded','attributes.rotation','iscrowd'])
    annotations_df[['x', 'y', 'width', 'height']]=pd.DataFrame(annotations_df['bbox'].to_list())

    merged_df = pd.merge(annotations_df, images_df, right_on='id',left_on="image_id", how='left')

    merged_df['x'] = merged_df['x'] /merged_df['width_y']
    merged_df['width_x'] = merged_df['width_x'] /merged_df['width_y']
    merged_df['y'] = merged_df['y'] /merged_df['height_y']
    merged_df['height_x'] = merged_df['height_x'] /merged_df['height_y']


    merged_df['new_id'] = merged_df.groupby('image_id').cumcount() + 1
    merged_df=merged_df.drop(columns=['id_x', 'image_id', 'bbox', 'id_y', 'width_y', 'height_y'])

    new_columns = {'category_id': 'class',
                'attributes.Team': 'team',
                'attributes.Ball_Possesion': 'Ball_Possesion',
                'attributes.Ball_Reciever': 'Ball_Reciever',
                'width_x': 'w',
                'height_x': 'h',
                'file_name': 'image_name',
                'new_id': 'OID'}

    merged_df = merged_df.rename(columns=new_columns)
    merged_df.loc[merged_df['class']==1,'class']=0
    merged_df.loc[merged_df['class']==2,'class']=1
    # merged_df.to_csv("mer.csv",index=False)
    dfs.append(merged_df)

label_df=pd.concat(dfs)
display(label_df)


,class,Ball_Possesion,Ball_Reciever,team,x,y,w,h,image_name,OID
0,0,0,0,0,0.000000,0.265278,0.031250,0.105556,image_vics_1344.png,1
1,0,1,0,1,0.344531,0.427778,0.034375,0.116667,image_vics_1344.png,2
2,0,0,0,0,0.314063,0.400000,0.034375,0.123611,image_vics_1344.png,3
3,1,NaN,NaN,NaN,0.402344,0.480556,0.010156,0.019444,image_vics_1344.png,4
4,0,0,0,0,0.507031,0.200000,0.014844,0.075000,image_vics_1344.png,5
...,...,...,...,...,...,...,...,...,...,...
3306,0,0,0,0,0.316406,0.386111,0.022656,0.090278,image_vimp_678.png,11
3307,0,0,0,2,0.789062,0.355556,0.021875,0.109722,image_vimp_678.png,12
3308,0,0,0,0,0.876563,0.268056,0.017188,0.084722,image_vimp_678.png,13
3309,0,0,0,1,0.784375,0.287500,0.017969,0.088889,image_vimp_678.png,14


In [5]:
# Get all val image filenames
IMAGE_PATH = 'downloads/SoccerPass/SoccerPass-COCO/images/val'
IMAGE_FILENAMES = [os.path.join(IMAGE_PATH, f) for f in os.listdir(IMAGE_PATH)] #[:10] #ONLY 10 IMAGES FOR TESTING, REMOVE LATER
len(IMAGE_FILENAMES)

207

In [6]:
# Object Detection 
import torch
from ultralytics import YOLO


# Build and load model (Yolov8l without pretraining, 1080 resolution)
Yolov8Detector = YOLO('runs/detect/yolov8l_soccerpass_fixed_res_1080_nopretrain/weights/best.pt')

device = 0 #0,1,2=cuda device 0,1,2 else 'cpu'

#Pass Data obtain Predictions
results = []
for i in range(len(IMAGE_FILENAMES)):
    r = Yolov8Detector.predict(source=IMAGE_FILENAMES[i], device=device)
    results += r
len(results)

OSError: /opt/conda/lib/python3.7/site-packages/torch/lib/../../nvidia/cublas/lib/libcublas.so.11: symbol cublasLtGetStatusString version libcublasLt.so.11 not defined in file libcublasLt.so.11 with link time reference

In [6]:
# Create dataframe from object detection predictions
detect_df = [['class', 'x', 'y', 'w', 'h', 'image_name', 'OID']]

for res in results:
    for i, box in enumerate(res.boxes):
        oid = i+1
        coords = box.xywhn.detach().cpu()
        cls = box.cls.detach().cpu()
        row = [int(cls.data[0]) ,coords[0][0].item(), coords[0][1].item(), coords[0][2].item(), coords[0][3].item(), os.path.split(res.path)[1], oid]
        detect_df.append(row)
headers = detect_df.pop(0)
detect_df = pd.DataFrame(detect_df, columns=headers)
detect_df

,class,x,y,w,h,image_name,OID
0,0,0.205118,0.705206,0.049804,0.151780,image_snjv_31.png,1
1,0,0.812973,0.379201,0.027491,0.109707,image_snjv_31.png,2
2,0,0.561093,0.598562,0.047636,0.124644,image_snjv_31.png,3
3,0,0.744281,0.913392,0.049667,0.170521,image_snjv_31.png,4
4,0,0.563893,0.410553,0.023120,0.113618,image_snjv_31.png,5
...,...,...,...,...,...,...,...
3373,0,0.168206,0.409499,0.024705,0.087617,image_snjv_26.png,12
3374,0,0.567300,0.332049,0.014766,0.074426,image_snjv_26.png,13
3375,0,0.550931,0.416630,0.016643,0.090706,image_snjv_26.png,14
3376,0,0.166430,0.322427,0.020103,0.068800,image_snjv_26.png,15


In [7]:
# Hungarian Matching between predicted boxes df and labels df. Will only be used for eval (inference doesnt need label)
import numpy as np
from scipy.optimize import linear_sum_assignment
from sklearn.metrics.pairwise import euclidean_distances

pd.set_option('mode.chained_assignment', None)

# select the common image names between the two DataFrames
common_image_names = set(label_df['image_name']).intersection(set(detect_df['image_name']))

# loop through each common image name and perform the matching
for image_name in common_image_names:
    # select the relevant rows from each DataFrame based on the image name
    labels_image = label_df[label_df['image_name'] == image_name]
    pred_image = detect_df[detect_df['image_name'] == image_name]
    
    # calculate the centroids of the labels objects
    labels_image['x_center'] = labels_image['x'] + (labels_image['w'] / 2)
    labels_image['y_center'] = labels_image['y'] + (labels_image['h'] / 2)
    labels_centroids = labels_image.loc[:, ['x_center', 'y_center']].values
    
    # get the centroids of the predicted objects
    pred_centroids = pred_image.loc[:, ['x', 'y']].values
    
    # calculate the pairwise Euclidean distances between the centroids
    distances = euclidean_distances(labels_centroids, pred_centroids)
    
    # use the Hungarian algorithm to find the optimal assignment of labels to preds
    row_ind, col_ind = linear_sum_assignment(distances)
    
    # loop through each assignment and append the predicted class, x, y, w, and h to the labels_df
    for i, j in zip(row_ind, col_ind):
        label_index = labels_image.index[i]
        pred_index = pred_image.index[j]
        label_df.loc[label_index, 'pred_class'] = detect_df.loc[pred_index, 'class']
        label_df.loc[label_index, 'pred_x'] = detect_df.loc[pred_index, 'x']
        label_df.loc[label_index, 'pred_y'] = detect_df.loc[pred_index, 'y']
        label_df.loc[label_index, 'pred_w'] = detect_df.loc[pred_index, 'w']
        label_df.loc[label_index, 'pred_h'] = detect_df.loc[pred_index, 'h']
        
display(label_df)

,class,Ball_Possesion,Ball_Reciever,team,x,y,w,h,image_name,OID,pred_class,pred_x,pred_y,pred_w,pred_h
0,0,0,0,0,0.000000,0.265278,0.031250,0.105556,image_vics_1344.png,1,0.0,0.015327,0.318150,0.030654,0.105568
1,0,1,0,1,0.344531,0.427778,0.034375,0.116667,image_vics_1344.png,2,0.0,0.366168,0.484811,0.047190,0.118679
2,0,0,0,0,0.314063,0.400000,0.034375,0.123611,image_vics_1344.png,3,0.0,0.331753,0.462112,0.035968,0.124128
3,1,NaN,NaN,NaN,0.402344,0.480556,0.010156,0.019444,image_vics_1344.png,4,1.0,0.407853,0.490848,0.010881,0.020293
4,0,0,0,0,0.507031,0.200000,0.014844,0.075000,image_vics_1344.png,5,0.0,0.514831,0.238202,0.015551,0.073689
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3306,0,0,0,0,0.316406,0.386111,0.022656,0.090278,image_vimp_678.png,11,0.0,0.327721,0.433267,0.021583,0.097270
3307,0,0,0,2,0.789062,0.355556,0.021875,0.109722,image_vimp_678.png,12,0.0,0.799344,0.411012,0.022984,0.109050
3308,0,0,0,0,0.876563,0.268056,0.017188,0.084722,image_vimp_678.png,13,0.0,0.885345,0.309862,0.017325,0.086714
3309,0,0,0,1,0.784375,0.287500,0.017969,0.088889,image_vimp_678.png,14,0.0,0.792633,0.332272,0.017939,0.091417


#### Player Clustering 



In [ ]:
import numpy as np
from clustimage import Clustimage

from matplotlib import image

import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter

import os
import sys

import cv2

In [ ]:
def pre_process_crop(image, bbox):
    H = image.shape[0]
    W = image.shape[1]

    x = int(bbox[0]*W)
    y = int(bbox[1]*H)
    w = int(bbox[2]*W)
    h = int(bbox[3]*H)
    
    #Extracting Crop from given image with bbox specs
    crop = image[y:y+h, x:x+w]
    crop = cv2.resize(crop, (20,40))

    #Applying filters to the crop
    crop = cv2.medianBlur(crop,3)
    crop = cv2.bilateralFilter(crop,5,30,30)
    
    return crop

In [ ]:
def formatPred(pred):
  y = np.unique(pred, return_counts=True)
  z = sorted(zip(y[1], y[0]), reverse=True)
  res = pred.copy()
  i=0
  for tup in z:
    idx_list = np.where(pred == tup[1])[0]
    
    # print(tup, idx_list)
    for idx in idx_list:
      res[idx] = i
    i+=1

  return fin

In [ ]:
def cluster(yolo_df):
  total_boxes = 0
  total_misclassified = 0
  
  groups = yolo_df.groupby("image_name")


  # loop through each group and apply the code snippet to each image
  for image_name, group in groups:
      image_path = None
      for path in paths:
          temp_path = os.path.join(path, image_name)
          if os.path.exists(temp_path):
              image_path = temp_path
              break
      
      if image_path is None:
          raise ValueError(f"Image file not found: {image_name}")

      img = image.imread(image_path)
      # plt.imshow(img)
      # plt.show()

      players_info = group[group['class'] == 1]

      true_labels = np.array(players_info['team'].tolist(), dtype=np.int32)
      num_cats = len(pd.unique(players_info['team'])) 

      player_boxes = []
      
      for index, row in players_info.iterrows():
        bbox = [row['x']-(row['w']/2), row['y']-(row['h']/2), row['w'], row['h']]
        crop = pre_process_crop(img, bbox)
        player_boxes.append(crop)

      player_boxes_reshaped = np.array(player_boxes).reshape(len(player_boxes),-1)
      player_boxes_reshaped = player_boxes_reshaped/255.0

      cl = Clustimage(method='pca',
                    embedding='tsne',
                    grayscale=False,
                    dim=(20,40),
                    params_pca={'n_components':0.95},
                    verbose=60)

      results = cl.fit_transform(player_boxes_reshaped,
                                cluster='agglomerative',
                                evaluate='silhouette',
                                metric='euclidean',
                                linkage='ward',
                                min_clust=num_cats,
                                max_clust=num_cats,
                                cluster_space='high')

      pred_labels = results['labels']
      
      preds = formatPred(pred_labels)

      i = 0
      for index, row in players_info.iterrows():
        if(row['class'] == 2):
          continue
        yolo_df.loc[index, 'pred_team'] = int(preds[i])
        i+=1
      print("name: {}, preds: {}, total boxes in : {}".format(image_name, preds, players_info.shape[0]))

  print("total_boxes: {} ,  total_images: {}".format(total_boxes, groups.shape[0]))
  return yolo_df




In [ ]:
label_df = cluster(label_df)

#### Perspective Transformation


In [1]:

from elements.perspective_transform import Perspective_Transform
from elements.assets import transform_matrix, detect_color
from elements.args import Args

import torch
import os
import cv2
import numpy as np
import sys

#Load Arguments for perspective transform model
opt = Args()

# Load model
pt_model = Perspective_Transform(opt)



def generate_perspective_transform(model, image_path->List, yolo_df):

    groups = yolo_df.groupby("image_name")

    # loop through each group and apply the code snippet to each image
    for image_name, group in groups:


        image_path = None
        for path in paths:
            temp_path = os.path.join(path, image_name)
            if os.path.exists(temp_path):
                image_path = temp_path
                break
        
        if image_path is None:
            raise ValueError(f"Image file not found: {image_name}")
    

        cap = cv2.imread(image_path)

        w = cap.shape[1] #1280
        h = cap.shape[0] #720

        main_frame = cap.copy()

        M, warped_image = perspective_transform.homography_matrix(main_frame)


        for index, row in group.iterrows():
            x_center = row['x'] #if x is center
            y_center = row['y'] + (row['h']/2)

            coords = transform_matrix(M, (x_center*w, y_center*h), (h, w), (68, 105))

            yolo_df.loc[index, 'gnd_x'] = coords[0]
            yolo_df.loc[index, 'gnd_y'] = coords[1]
        
        return yolo_df

paths =[]

transformed_df = generate_perspective_transform( pt_model,paths, label_df)









# For image and all bbox+ID => return ID +x+y coordinates

SyntaxError: invalid syntax (2158458125.py, line 22)

In [ ]:
#Ball Possession Identification

#min distance between ball and player

In [ ]:
#Reciever Identification

#Convert ID+x+y+team+bp+br 